In [ ]:
#@title Install and Auto restart
!pip install accelerate==0.32.1
!pip install huggingface-hub==0.24.6
!pip install Pillow==10.4.0
# !pip install torch==2.3.1
# !pip install torchvision==0.18.1
!pip install transformers==4.44.0
!pip install einops==0.8.0
!pip install gradio==4.36.1
from IPython.display import clear_output
clear_output()
import time
time.sleep(6)
import os
os.kill(os.getpid(), 9)

In [1]:
#@title <-- Play the audio { display-mode: "form" }

# !git clone https://github.com/vikhyat/moondream.git
# %cd /content/moondream
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from PIL import Image

# model_id = "vikhyatk/moondream2"
# revision = "2024-03-13"
# model = AutoModelForCausalLM.from_pretrained(
#     model_id, trust_remote_code=True, revision=revision
# )
# tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)
# from PIL import Image
# image = Image.open('/content/monalisa.jpg')
# image = Image.open('/content/monalisa.jpg')
# enc_image = model.encode_image(image)
# print(model.answer_question(enc_image, "Describe this image.", tokenizer))
%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [2]:
#@title Downlod and Config MoonDream
import torch
import re
import time
# from moondream.moondream import detect_device, LATEST_REVISION
from threading import Thread
from transformers import TextIteratorStreamer, AutoTokenizer, AutoModelForCausalLM
from PIL import Image


import torch
if torch.cuda.is_available():
    use_cpu=False
else:
    use_cpu=True
# Specify whether to use CPU or GPU
# use_cpu = True  # Change to True if you want to use CPU

if use_cpu:
  device = torch.device("cpu")
  dtype = torch.float32
else:
  device=torch.device("cuda")
  dtype=torch.float16
print("Using device:", device)

# revision = "2024-03-13"
revision = "2024-08-26"

model_id = "vikhyatk/moondream2"
# tokenizer = AutoTokenizer.from_pretrained(model_id, revision=LATEST_REVISION)
# moondream = AutoModelForCausalLM.from_pretrained(
#     model_id, trust_remote_code=True, revision=LATEST_REVISION
# ).to(device=device, dtype=dtype)

tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

moondream = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
).to(device=device, dtype=dtype)
moondream.eval()
from IPython.display import clear_output
clear_output()

In [3]:
#@title utils for graido
def answer_question(image, prompt):
    image_embeds = moondream.encode_image(image)
    streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)
    thread = Thread(
        target=moondream.answer_question,
        kwargs={
            "image_embeds": image_embeds,
            "question": prompt,
            "tokenizer": tokenizer,
            "streamer": streamer,
        },
    )
    thread.start()

    buffer = ""
    for new_text in streamer:
        clean_text = re.sub("<$|END$", "", new_text)
        buffer += clean_text
        yield buffer.strip("<END")
def get_answer(prompt,image):
  answer = []
  for text in answer_question(image, prompt):
      answer.append(text)

  if len(answer) == 0:
      answer.append("Nothing Found")

  return answer[-1]
import uuid
def random_image_name():
  random_uuid = uuid.uuid4()
  image_extension = ".jpg"
  random_image_name = str(random_uuid)[:8] + image_extension
  return random_image_name
import shutil
import os
import uuid
from PIL import Image

if not os.path.exists("/content/upload"):
    os.mkdir("/content/upload")

def process_upload_image(prompt, gradio_image):
    print(gradio_image)
    print(type(gradio_image))
    try:
        # Handle PIL format image
        image = Image.open(gradio_image)
        # Generate a random image name using UUID
        image_name = random_image_name()
        # Save the image to the upload directory
        copy_image_path = os.path.join("/content/upload", image_name)
        print(f"Upload Image Saved at {copy_image_path}")
        image.save(copy_image_path)
        answer=get_answer(prompt,image)
        return answer
    except Exception as e:
        print(f"Error: {e}")
        copy_image_path=f"/content/upload/{random_image_name()}"
        gradio_image.save(copy_image_path)
        print(f"Upload Image Saved at {copy_image_path}")
        answer=get_answer(prompt,gradio_image)
        return answer

# # Example usage:
# gradio_image_path = "/content/monalisa.jpg"  # Replace with the actual path to your image
# prompt = "What's going on? Respond with a single sentence."
# process_upload_image(prompt, gradio_image_path)
# gradio_image_path = Image.open("/content/monalisa.jpg")
# process_upload_image(prompt, gradio_image_path)

In [ ]:
#@title Gradio API Interface
username = 'admin'  # @param {type: "string"}
password = 'admin'  # @param {type: "string"}
Debug = True  # @param {type: "boolean"}

import gradio as gr
image_inputs=[gr.Textbox(label="Write Prompt",value="What's going on? Respond with a single sentence."),gr.Image(type='pil',label="Upload a Image")]
# image_outputs=[gr.Textbox(label="Result")]
image_outputs=[gr.Textbox(label="Response")]

app_demo = gr.Interface(fn=process_upload_image, inputs=image_inputs,outputs=image_outputs , title="MoonDream")
app_demo.queue().launch(share=True,debug=Debug,auth=(username, password))